In [16]:
    # @autor: assispaivaneto
    # @data: 20/11/2021
    # @objetivo: pre-processmento de um dataset

Existem diversas maneiras de um dado ser criado ou alterado, com isso o modo como conhecemos
algum determinado tipo de dado oscila de forma incomum, principalmente quando trabalhamos com
dados textuais, já que a internet através das redes sociais, entrega diversas possibilidades
para as pessoas escrevem conforme sua vontade.

Conforme a situação apresentada, sempre temos que nos atentar quando surgir a necessidade de ser 
feito um processamento de dados, por conta que, na sua forma "natural" podem atrapalhar ou atrasar 
o processo que vai ser realziado.

É por conta disso, que surge a necessidade do pré-processamento de dados. E desse modo, quando estamos 
trabalhando com textos, o pré-processamento acontece nas seguintes etapas:
    1 - tornar todas as palaras minúsculas;
    2 - excluir as contrações existentes;
    3 - remover as pontuações, números e espaços em branco;
    4 - tokenizar: processo onde o texto é dividido em partes menores;
    5 - lematizar: processo onde consideramos apenas o lema das palavras.
    
Através desse processo, são eliminados alguns itens que não tem relevância para o processamento
final, fazendo com que, essa operação ocorra de maneira mais objetiva e consequentemente mais rápida,
clara e obtenha um resultado mais real.

 <font size="3"> Importação de bibliotecas e métodos que vão ser utilizados </font>

In [10]:
import numpy as np
import pandas as pd
import pandas as pd
import nltk
import spacy
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

 <font size="3"> Carregando a base de dados </font>

In [3]:
data = pd.read_csv("/home/assis/Documentos/Programação/Python/Projeto final/train.csv")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


 <font size="3"> Realocando os dados do dataset </font>

In [4]:
toxic_mask = (data[data.columns[2:]] > 0).any(axis=1)
data = {"text": data["comment_text"], "toxic": np.zeros(data.shape[0], dtype=np.int64)}
data["toxic"][toxic_mask] = 1
data = pd.DataFrame(data)

data.head() 

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


 <font size="3"> Funções para o pré-processamento dos dados </font>

In [14]:
nlp = spacy.load("en_core_web_sm")

def decontracted(phrase):
    '''
    É comum encontrarmos contrações nas palavras, principalmente quando
    estamos trabalhando com textos em inglês (o que acontece com grande
    frequência devido a influência e alcance desse idioma).
    Desse modo, decontracted() é uma função que "elimina" as contrações 
    existentes.
    '''
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def lemmatizeText(text):
    '''
    A lematização é um processo que reduz a palavra a sua forma base.
    De modo que, todas as palavras que possuem a mesma base, sejam 
    agrupadas. 
    Como exemplo temos as palvras "brincar", "brincou",
    "brincava", todas formas da palavra "brinca", dessa forma, a pala-
    vra "brinca" é o lema de todas as anteriores.
    Com isso, a função lemmatizeText(), tem como objetivo aplicar o 
    processo de lematização que foi mencionado.
    '''
    doc = nlp(text)
    lemmed = [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]
    return ' '.join(lemmed)

def preProcessamento(x):
    '''
    A função preProcessamento() realiza de forma geral o pré-processa-
    mento dos dados. Para que isso seja possível, ela faz uso de outras
    funções já definidas e de alguns métodos.
    '''
    x = x.lower()
    x = decontracted(str(x))
    x = re.sub(r'[^\w\s]', ' ', x)
    x = re.sub(r'[0-9]+', '', x)
    x = nltk.word_tokenize(x)
    x = [word for word in x if not word in stopwords.words()]
    x = lemmatizeText(' '.join(x))
    return x

 <font size="3"> Realizando o pré-procesamento dos dados </font>

In [18]:
texts = []

for i, text in enumerate(data['text']):
    text = preProcessamento(text)
    texts.append(text)
    print(f'{i+1}/{data.shape[0]}', end='\r')
    
newData = {'text': texts, 'toxic': data['toxic'].tolist()}
newData = pd.DataFrame(newData)
newData.to_json('PreProcessData.json')